In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

WLmodel = cobra.io.read_sbml_model(join(data_dir,'Models/WLPathway_bifurcating.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


To properly compare all results for the different substrate and product combinations, the model is constrained by the flux through CODH_ACS, so to equal Acetyl-CoA.

In [3]:
WLmodel.reactions.CODH_ACS.bounds = 1,1

In [4]:
WLmodel.metabolites.get_by_id("10fthf_c").name

'10-Formyltetrahydrofolate'

In [6]:
WLmodel.metabolites.h2_e.name  = "CO",
WLmodel.metabolites.h2o_e.name = "H2O",
WLmodel.metabolites.co2_e.name = "CO2",
WLmodel.metabolites.co_e.name = "CO",
WLmodel.metabolites.pi_e.name = "Pi", # Inorganic phosphate
WLmodel.metabolites.h_e.name = "H+",
WLmodel.metabolites.ac_e.name = "Acetate",
WLmodel.metabolites.etoh_e.name = "Ethanol",
WLmodel.metabolites.btd_RR_e.name = "Butanediol",
WLmodel.metabolites.but_e.name = "Butyrate",
WLmodel.metabolites.btoh_e.name = "Butanol",
WLmodel.metabolites.h2_c.name = "CO",
WLmodel.metabolites.h2o_c.name = "H2O",
WLmodel.metabolites.co2_c.name = "CO2",
WLmodel.metabolites.co_c.name = "CO",
WLmodel.metabolites.ac_c.name = "Acetate",
WLmodel.metabolites.h_c.name = "H+",
WLmodel.metabolites.etoh_c.name = "Ethanol",
WLmodel.metabolites.btd_RR_c.name = "Butanediol",
WLmodel.metabolites.but_c.name = "Butyrate",
WLmodel.metabolites.btoh_c.name = "Butanol",
WLmodel.metabolites.fdxr_42_c.name = "Ferredoxin red.",
WLmodel.metabolites.fdxo_42_c.name = "Ferredoxin ox.",
WLmodel.metabolites.for_c.name = "Formate",
WLmodel.metabolites.atp_c.name = "ATP",
WLmodel.metabolites.thf_c.name = "THF", # 5,6,7,8-Tetrahydrofolate
WLmodel.metabolites.get_by_id("10fthf_c").name = "10-FTHF", # 10-Formyltetrahydrofolate
WLmodel.metabolites.adp_c.name = "ADP",
WLmodel.metabolites.pi_c.name = "Pi",
WLmodel.metabolites.methf_c.name = "METHF", #5,10-Methenyltetrahydrofolate
WLmodel.metabolites.mlthf_c.name = "MLTHF", #5,10-Methylenetetrahydrofolate
WLmodel.metabolites.nadp_c.name = "NADP", # Nicotinamide adenine dinucleotide phosphate
WLmodel.metabolites.nadph_c.name = "NADPH", # Nicotinamide adenine dinucleotide phosphate - reduced
WLmodel.metabolites.nadh_c.name = "NADH", # Nicotinamide adenine dinucleotide - reduced
WLmodel.metabolites.get_by_id("5mthf_c").name = "5MTHF", # 5-Methyltetrahydrofolate
WLmodel.metabolites.nad_c.name = "NAD", # Nicotinamide adenine dinucleotide
WLmodel.metabolites.cfesp_c.name = "CFESP", # Corrinoid Iron sulfur protein
WLmodel.metabolites.mecfsp_c.name = "MECFSP", # Methylcorrinoid iron sulfur protein
WLmodel.metabolites.coa_c.name = "CoA", # Coenzyme A
WLmodel.metabolites.accoa_c.name = "Acetyl_CoA",
WLmodel.metabolites.actp_c.name = "Acetyl P",
WLmodel.metabolites.acald_c.name = "Acetaldehyde",
WLmodel.metabolites.pyr_c.name = "Pyruvate",
WLmodel.metabolites.alac__S_c.name = "Acetolactate",
WLmodel.metabolites.actn__R_c.name = "Acetoin",
WLmodel.metabolites.aacoa_c.name = "Acetoacetyl-CoA",
WLmodel.metabolites.get_by_id("3hbcoa_c").name = "3HB-CoA", #(S)-3-Hydroxybutanoyl-CoA
WLmodel.metabolites.b2coa_c.name = "Crotonoyl-CoA",
WLmodel.metabolites.btcoa_c.name = "Butanoyl-CoA",
WLmodel.metabolites.btal_c.name = "Butanal",
WLmodel.metabolites.butpi_c.name = "Butanoyl P",
WLmodel.metabolites.acac_c.name = "Acetoacetate",
WLmodel.metabolites.act_c.name = "Acetone",
WLmodel.metabolites.act_e.name = "Acetone"

Define carboxydotrophic and homoacetogenic media, Acetyl-CoA is constrained, so substrates are only absent or present and uptake rates are unconstrained

In [7]:
medium_carb = WLmodel.medium
medium_carb['EX_co_e'] = 1000
medium_carb['EX_co2_e'] = 0
medium_carb['EX_h2_e'] = 0

medium_hom = WLmodel.medium
medium_hom['EX_co_e'] = 0
medium_hom['EX_co2_e'] = 1000
medium_hom['EX_h2_e'] = 1000

In [8]:
objectives = pd.Series(["EX_ac_e","EX_etoh_e","EX_act_e","EX_but_e","EX_btoh_e","EX_btd_RR_e"])
formulas = pd.Series(["CH3COOH","C2H5OH","C3H6O","C4H7O2","C4H9OH","C4H10O2"], index=objectives)
Cmol =  pd.Series([2,2,3,4,4,4], index=objectives)

for x in objectives:
    print(WLmodel.reactions.get_by_id(x).name)
    WLmodel.objective = x
    with WLmodel:
        WLmodel.medium = medium_carb
        WLmodel.optimize()
        print(f" carb: {round(WLmodel.reactions.EX_co_e.flux,2)} CO + {round(WLmodel.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodel.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodel.slim_optimize(),2)} {formulas[x]} + {round(WLmodel.reactions.EX_h_e.flux,2)} H+ + {round(WLmodel.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodel.reactions.SK_atp_c.flux/WLmodel.reactions.EX_co_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodel.slim_optimize()*Cmol[x]/WLmodel.reactions.EX_co_e.flux,2))

    with WLmodel:
        WLmodel.medium = medium_hom
        WLmodel.optimize()
        print(f" hom: {round(WLmodel.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodel.reactions.EX_h2_e.flux,2)} H2 + {round(WLmodel.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodel.slim_optimize(),2)} {formulas[x]} + {round(WLmodel.reactions.EX_h_e.flux,2)} H+ + {round(WLmodel.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodel.reactions.SK_atp_c.flux/WLmodel.reactions.EX_h2_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodel.slim_optimize()*Cmol[x]/WLmodel.reactions.EX_h2_e.flux,2), "\n")

Acetate exchange
 carb: -4.0 CO + -0.33 H2O + 2.0 CO2 + 1.0 CH3COOH + -0.67 H+ + 1.67 ATP
   ATP/Electron donor:  0.42
   Product/Electron donor:  0.5
 hom: -2.0 CO2 + -4.0 H2 + 2.33 H2O + 1.0 CH3COOH + 0.67 H+ + 0.33 ATP
   ATP/Electron donor:  0.08
   Product/Electron donor:  0.5 

Ethanol exchange
 carb: -6.0 CO + -0.67 H2O + 4.0 CO2 + 1.0 C2H5OH + -2.33 H+ + 2.33 ATP
   ATP/Electron donor:  0.39
   Product/Electron donor:  0.33
 hom: -2.0 CO2 + -6.0 H2 + 3.33 H2O + 1.0 C2H5OH + -0.33 H+ + 0.33 ATP
   ATP/Electron donor:  0.06
   Product/Electron donor:  0.33 

Acetone exchange
 carb: -4.0 CO + -0.5 H2O + 2.5 CO2 + 0.5 C3H6O + -1.0 H+ + 1.0 ATP
   ATP/Electron donor:  0.25
   Product/Electron donor:  0.38
 hom: -1.5 CO2 + -4.0 H2 + 2.17 H2O + 0.5 C3H6O + 0.33 H+ + -0.33 ATP
   ATP/Electron donor:  -0.08
   Product/Electron donor:  0.38 

Butyrate (n-C4:0) exchange
 carb: -5.0 CO + -0.17 H2O + 3.0 CO2 + 0.5 C4H7O2 + -1.33 H+ + 1.83 ATP
   ATP/Electron donor:  0.37
   Product/Electron

## Closer look, change x within [0:5] to view different product

In [9]:
x = 0

WLmodel.objective = objectives[x]

#Medium: choose one by commenting the other out
WLmodel.medium = medium_carb
#WLmodel.medium = medium_hom

solution = WLmodel.optimize()

Builder(model = WLmodel, map_json = join(data_dir,'Models/EscherMaps/WLPathway_Bifurcating.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': 0.0, 'EX_h2o_e': -0.3333333333333335, 'EX_co2_e': 2.0, 'EX_co_e': -4.0, 'EX_…